In [24]:
import requests
import pandas as pd

API_KEY = ""
BASE_URL = "https://yfapi.net"

def get_basic_stock_data(symbols):
    """Fetch basic stock data (Task 1)"""
    endpoint = f"{BASE_URL}/v6/finance/quote"
    headers = {'x-api-key': API_KEY}

    response = requests.get(endpoint, headers=headers, params={"symbols": ",".join(symbols)})

    if response.status_code == 200:
        result = response.json()
        if 'quoteResponse' in result and 'result' in result['quoteResponse']:
            stocks = []
            for stock in result['quoteResponse']['result']:
                stocks.append({
                    "Stock Ticker": stock.get('symbol', 'N/A'),
                    "Company Name": stock.get('longName', 'N/A'),
                    "Current Market Price": format_currency(stock.get('regularMarketPrice', 'N/A'))
                })
            return stocks
    print(f"Error fetching data: {response.status_code}")
    return []

def get_additional_metrics(symbols):
    """Fetch additional stock data (Task 2.1) - 52 Week High, 52 Week Low, ROA"""
    endpoint = f"{BASE_URL}/v11/finance/quoteSummary/"
    headers = {'x-api-key': API_KEY}

    metrics = []
    for symbol in symbols:
        response = requests.get(f"{endpoint}{symbol}", headers=headers, params={"modules": "summaryDetail,financialData"})

        if response.status_code == 200:
            try:
                result = response.json()
                if 'quoteSummary' in result and result['quoteSummary']['result']:
                    financial_data = result['quoteSummary']['result'][0]
                    metrics.append({
                        'Symbol': symbol,
                        '52 Week High': financial_data['summaryDetail'].get('fiftyTwoWeekHigh', 'N/A'),
                        '52 Week Low': financial_data['summaryDetail'].get('fiftyTwoWeekLow', 'N/A'),
                        'ROA': round(financial_data['financialData'].get('returnOnAssets', 'N/A'), 4) if isinstance(financial_data['financialData'].get('returnOnAssets'), float) else 'N/A'
                    })
                else:
                    print(f"⚠️ No financial data found for {symbol}")
            except (KeyError, TypeError) as e:
                print(f" Error processing data for {symbol}: {e}")
        else:
            print(f" Error fetching metrics for {symbol}: {response.status_code}")


    return pd.DataFrame(metrics) if metrics else pd.DataFrame()

def format_currency(value):
    """Format numeric values as currency"""
    if isinstance(value, (int, float)):
        return f"${value:,.2f}"
    return value

def main():
    # Task 1:
    symbols = input("Enter stock symbols (comma-separated): ").upper().split(',')
    basic_data = get_basic_stock_data(symbols)

    print("\nBasic Stock Data:")
    for stock in basic_data:
        print(f"{stock['Stock Ticker']} | {stock['Company Name']} | {stock['Current Market Price']}")

    # Task 2.1:
    print("\nAdditional Metrics (52w High, 52w Low, ROA):")
    metrics_df = get_additional_metrics(symbols)
    print(metrics_df.to_string(index=False))

if __name__ == "__main__":
    main()

Enter stock symbols (comma-separated): AAPL,MSFT

Basic Stock Data:
AAPL | Apple Inc. | $241.84
MSFT | Microsoft Corporation | $396.99

Additional Metrics (52w High, 52w Low, ROA):
Symbol                     52 Week High                      52 Week Low ROA
  AAPL  {'raw': 260.1, 'fmt': '260.10'} {'raw': 164.08, 'fmt': '164.08'} N/A
  MSFT {'raw': 468.35, 'fmt': '468.35'} {'raw': 385.58, 'fmt': '385.58'} N/A
